In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

titanic_df = pd.read_csv('./data/train.csv')
titanic_df.head(3)

# 학습 정보 데이터 출력
print('\n ### 학습데이터 정보 ### \n')
print(titanic_df.info())
titanic_df


 ### 학습데이터 정보 ### 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### 결측치 정제
- 머신러닝 알고리즘은 null값을 허용하지 않기에 결측치를 처리한다

In [ ]:
titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True) # Age컬럼의 결측치 자리에 Age의 평균값 삽입
titanic_df['Cabin'].fillna('N', inplace=True) # Cabin 컬럼의 결측치 자리에 'N' 삽입
titanic_df['Embarked'].fillna('N', inplace=True) # Embarked 컬럼의 결측치 자리에도 'N' 삽입
print('데이터 세트 Null값 개수 = ', titanic_df.isna().sum().sum()) 

### 문자열 피쳐 정제하기


In [ ]:
print('Sex값 분포 : \n', titanic_df['Sex'].value_counts())
print('\n Cabin값 분포 : \n', titanic_df['Cabin'].value_counts())
print('\n Embarked값 분포 : \n', titanic_df['Embarked'].value_counts())

# Sex와 Embarked는 문제가 없으나, Cabin의 경우 C23 C25 C27과 같이 여러 Cabin이 함께 포함되어 적힌 값이 4개가 있으므로
# Cabin속성의 경우에는 맨 앞글자만 추출하기로 하자.
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
titanic_df['Cabin'].head(3)

### 어떤 유행의 승객이 생존확률이 높았을까?
- 성별에 따른 생존자 수 비교하기

In [ ]:
titanic_df.groupby(['Sex', 'Survived'])['Survived'].count()
# 남자의 경우 577명 중 109명이 생존하여 18.8%의 생존율을 가지고
# 여자의 경우 314명 중 233명이 생존하여 74.2%의 생존율을 가진다. 이를 barplot으로 확인해보자
sns.barplot(x = 'Sex', y = 'Survived', data=titanic_df)

- 부유한 정도에 따른 생존비율

In [ ]:
# 부유함의 척도를 알아볼 수 있는 컬럼은 객실이다. 효율성을 추가하기 위해 성별을 추가해서 같이보자.
sns.barplot(x = 'Pclass', y = 'Survived', hue = 'Sex', data = titanic_df)

- 나이에 따른 생존비율

In [ ]:
# 나이에 따라서 구분값을 반환하는 함수를 작성해보자
def get_category(age):
    cat = ''
    if age<=-1: cat = 'Unknown'
    elif age <=5: cat = 'Baby'
    elif age <= 12: cat = 'Child'
    elif age <= 18: cat = 'Teenager'
    elif age <= 25: cat = 'Student'
    elif age <= 35: cat = 'Young Adult'
    elif age <= 60: cat = 'Adult'
    else: cat = 'Elderly'
    return cat

# 그래프 크기 설정
plt.figure(figsize=(10, 6))

# X축의 값을 순차적으로 표시하기 위해 그래프에 그려질 X축 이름 리스트 생성
group_name= ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Elderly']

# lambda식에 위에서 생성한 함수를 반환값으로 지정
titanic_df['Age_cat'] = titanic_df['Age'].apply(lambda x : get_category(x))
sns.barplot(x = 'Age_cat', y = 'Survived', hue = 'Sex', data = titanic_df, order = group_name)
titanic_df.drop('Age_cat', axis = 1, inplace=True)


# 이로서 Sex, pclass, Age가 생존에 있어 중요한 컬럼들임을 알게되었다.

### 문자열 카테고리 피처 인코딩
- labelencoder를 활용해 Cabin, Sex, Embarked 인코딩

In [3]:
from sklearn.preprocessing import LabelEncoder
def encode_features(dataDF):
    features = ['Cabin', 'Sex', 'Embarked'] 
    for feature in features:
        le = LabelEncoder()
        le = le.fit(dataDF[feature]) # Cabin과 Sex와 Embarked가 for loop안에서 차례대로 학습된다
        dataDF[feature] = le.transform(dataDF[feature]) # 차례대로 encoding 된 결과들을
    
    return dataDF # 한꺼번에 return

titanic_df = encode_features(titanic_df)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,147,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,81,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,147,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,55,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,147,2


### 피처 가공함수 만들기

In [5]:
# 결측치 처리함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace = True)
    df['Cabin'].fillna('N', inplace = True)
    df['Embarked'].fillna('N', inplace = True)
    df['Fare'].fillna(0, inplace = True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis = 1, inplace = True)
    return df

# 레이블 인코딩 수행하기
from sklearn.preprocessing import LabelEncoder

def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked'] 
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature]) # Cabin과 Sex와 Embarked가 for loop안에서 차례대로 학습된다
        df[feature] = le.transform(df[feature]) # 차례대로 encoding 된 결과들을
    
    return df # 한꺼번에 retur

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [7]:
# 원본 데이터를 재로딩하고, 피처 데이터 세트와 레이블 데이터 세트 추출
titanic_df = pd.read_csv('./data/train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis = 1)

X_titanic_df = transform_features(X_titanic_df)
X_titanic_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.000000,1,0,7.2500,7,3
1,1,0,38.000000,1,0,71.2833,2,0
2,3,0,26.000000,0,0,7.9250,7,3
3,1,0,35.000000,1,0,53.1000,2,3
4,3,1,35.000000,0,0,8.0500,7,3
...,...,...,...,...,...,...,...,...
886,2,1,27.000000,0,0,13.0000,7,3
887,1,0,19.000000,0,0,30.0000,1,3
888,3,0,29.699118,1,2,23.4500,7,3
889,1,1,26.000000,0,0,30.0000,2,0


In [ ]:
# 데이터 분리하기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size = 0.2, random_state=11)

### RandomForest, DecisionTreeClassifier, LogisticRegression으로 예측하기

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 결정트리, random forest, 로지스틱 회귀를 위한 사이킷런 classifier 클래스 생성
log = LogisticRegression(solver='liblinear')  # 알고리즘들 중에 하나를 선택한다.
                                              # solver='liblinear'는 소량의 데이터를 다룰때 적합하고, solver='saga'는 대량의 데이터를 다룰 때 월등히 좋다.
rf = RandomForestClassifier(random_state=11)
dtc = DecisionTreeClassifier(random_state=11)

# DecisionTreeClassifier 학습/예측/평가
dtc.fit(X_train, y_train)
dt_pred = dtc.predict(X_test)
print('DecisionTreeClassifier 정확도 : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

# RandomForest 학습/예측/평가
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print('RandomForest 정확도 : {0:.4f}'.format(accuracy_score(y_test, rf_pred)))

# LogisticRegression 학습/예측/평가
log.fit(X_train, y_train)
log_pred = log.predict(X_test)
print('LogisticRegression 정확도 : {0:.4f}'.format(accuracy_score(y_test, log_pred)))

### KFold 교차검증 실시

In [ ]:
from sklearn.model_selection import KFold
def exec_kfold(clf, folds = 5):
    # 폴드 세트를 5개인 kfold객체를 생성하는 함수이다. 폴드 수만큼 예측결과 저장을 위한 리스트 객체를 생성한다
    kfold = KFold(n_splits=5)
    scores = []

    # KFold교차 검증 실시
    for iter_count, (train_index, test_index) in enumerate(kfold.split(X_titanic_df)):
        # X_titanic_df 데이터에서 교차 검증별로 학습과 검증 데이터를 가리키는 index 생성
        X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
        y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]
        # Classifier 학습, 예측, 정확도 계산
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print('교차 검증 {0} 정확도 : {1:.4f}'.format(iter_count+1, accuracy))

    # 5개의 fold에서의 평균 정확도 계산
    mean_score = np.mean(scores)
    print('평균 정확도 : {0:.4f}'.format(mean_score))

# 함수 호출
exec_kfold(dtc, folds=5)

### cross_val_score API를 통한 교차검증
- KFold와 값이 다르게 나오는 이유는, cross_val_score는 StrarifiedKFold를 이용해 교차검증을 진행하기 때문이다.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dtc, X_titanic_df, y_titanic_df, cv=5)
for iter_count, accuracy in enumerate(scores):
    print('교차검증 {0} 정확도 : {0:.4f}'.format(iter_count, np.mean(scores)))
print('평균 정확도 : {0:.4f}'.format(np.mean(scores)))

### GridSearchCV를 이용한 DecisionTreeClassifier의 최적 하이퍼 파라미터를 찾고 예측 성능 측정해보기

In [43]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth':[2, 3, 5, 10],
    'min_samples_split':[2, 3, 5],
    'min_samples_leaf':[1, 5, 8]
}

grid_dclf = GridSearchCV(dtc, param_grid=parameters, scoring='accuracy', cv=5)
grid_dclf.fit(X_train, y_train)

print('GridSearchCV 최적 하이퍼 파라미터 : ',grid_dclf.best_params_)
print('GridSearchCV 최고 정확도 : {0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행
dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)
print('테스트 세트에서의 DecisionTreeClassifier 정확도 : {0:.4f}'.format(accuracy))

GridSearchCV 최적 하이퍼 파라미터 :  {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.7992
테스트 세트에서의 DecisionTreeClassifier 정확도 : 0.8715
